# Classification
In this notebook we classify the CT/PET scans of lung cancer cases by tumor type. 
We are doing this as a baseline task to validate our data loading pipeline.
Most code is taken from [this tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from nscl_dataset import NSCLDataSet

cudnn.benchmark = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.cuda.list_gpu_processes()

'pynvml module not found, please install pynvml'

In [3]:
ds = NSCLDataSet()

In [13]:
next(iter(ds))[0].shape

torch.Size([3, 512, 512])

# Defining datasets (train/validation) 

In [9]:
def test(x):
    # print(x.shape)
    return x

preprocess = transforms.Compose([
    test,
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trainSet, valSet = random_split(NSCLDataSet(transform=preprocess), [0.8, 0.2])
dataloaders = {
    "train": torch.utils.data.DataLoader(trainSet, batch_size=4, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(valSet, batch_size=4, shuffle=True, num_workers=4)
}

class_names = ["Adenocarcinoma", "Small Cell Carcinoma", "Large Cell Carcinoma", "Squamous Cell Carcinoma"]

dataset_sizes = {"train": len(trainSet),"val": len(valSet)}

# Defining model 

In [10]:

resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# freeze all layers but last
for param in resnet.parameters():
    param.requires_grad = False

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, len(class_names))

Using cache found in /dhc/home/franz.sauerwald/.cache/torch/hub/pytorch_vision_v0.10.0
/dhc/home/franz.sauerwald/conda3/envs/advanced-ml/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/dhc/home/franz.sauerwald/conda3/envs/advanced-ml/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Setup training

In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for idx, (inputs, labels) in enumerate(dataloaders[phase]):
                if idx % 50 == 0:
                    print(f"Batch {idx}/{len(dataloaders[phase])}")
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Define training parameters

In [12]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Run training

In [17]:
resnet = train_model(resnet, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
Batch 0/6077
Batch 50/6077
Batch 100/6077
Batch 150/6077
Batch 200/6077
Batch 250/6077
Batch 300/6077
Batch 350/6077
Batch 400/6077
Batch 450/6077
Batch 500/6077
Batch 550/6077
Batch 600/6077
Batch 650/6077
Batch 700/6077
Batch 750/6077
Batch 800/6077
Batch 850/6077
Batch 900/6077
Batch 950/6077
Batch 1000/6077
Batch 1050/6077
Batch 1100/6077
Batch 1150/6077
Batch 1200/6077
Batch 1250/6077
Batch 1300/6077
Batch 1350/6077
Batch 1400/6077
Batch 1450/6077
Batch 1500/6077
Batch 1550/6077
Batch 1600/6077
Batch 1650/6077
Batch 1700/6077
Batch 1750/6077
Batch 1800/6077
Batch 1850/6077
Batch 1900/6077
Batch 1950/6077
Batch 2000/6077
Batch 2050/6077
Batch 2100/6077
Batch 2150/6077
Batch 2200/6077
Batch 2250/6077
Batch 2300/6077
Batch 2350/6077
Batch 2400/6077
Batch 2450/6077
Batch 2500/6077
Batch 2550/6077
Batch 2600/6077
Batch 2650/6077
Batch 2700/6077
Batch 2750/6077
Batch 2800/6077
Batch 2850/6077
Batch 2900/6077
Batch 2950/6077
Batch 3000/6077
Batch 3050/6077
Batch 310

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1